In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
#hide
from fastbook import *

# Collaborative Filtering Deep Dive

## A First Look at the Data

In [1]:
from fastai.collab import *
from fastai.tabular.all import *

In [2]:
path = Path('/notebooks/fastai_course_p1/Lesson7/ml-100k/')

In [3]:
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names=['user','movie','rating','timestamp'])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
movies = pd.read_csv(path/'u.item',  delimiter='|', encoding='latin-1',
                     usecols=(0,1), names=('movie','title'), header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [5]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [47]:
class DotProduct(Module):
    def __init__(self, n_movies, n_users, emb_depth):
        self.n_movies = n_movies
        self.n_users = n_users
        
        self.m_movies = torch.randn(size=(n_movies, emb_depth))
        self.m_users = torch.randn(size=(n_users, emb_depth))
    
    def forward(self, x):
        movie_embs = self.m_movies[x[:,0]]
        user_embs = self.m_users[x[:,1]]
        
        res = (users * movies).sum(dim=1)
        
        return res

In [48]:
x = ratings[['movie', 'user']].to_numpy()

In [49]:
x

array([[ 242,  196],
       [ 242,   63],
       [ 242,  226],
       ...,
       [1637,  655],
       [1630,  655],
       [1641,  655]])

In [50]:
n_users = ratings['user'].nunique()
n_movies = ratings['movie'].nunique()

In [51]:
a = ratings['user'].unique()
a.sort()
a

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [52]:
n_users

943

In [56]:
model = DotProduct(n_movies, n_users, 5)